### Phát biểu bài toán:

-   Phân tích mức độ ảnh hưởng của phần cứng và thương hiệu đến giá của điện thoại di động

### Nguồn Dữ liệu:

-   Dữ liệu được thu thập từ trang web:
    -   https://mobilecity.vn/dien-thoai
    -   https://cellphones.com.vn/mobile.html

Phân tích dữ liệu nhằm khảo sát tính khả thi cho việc xây dựng mô hình dự đoán biến mục tiêu (target variable) "Giá - Price" từ các biến/đặc trưng phần cứng và thương hiệu Xi (i=1..N).
Vì biến đăng trưng Y là biến số liên tục (continuous variable) -> việc mô hình hóa là bài toán hồi quy (regression).

### Các bước thực hiện:

-   Bước 1: Khai thác dữ liệu (Data Exploration)
-   Bước 2: Tiền xử lý dữ liệu (Data Preprocessing)
-   Bước 3: Phân tích dữ liệu (Data Analysis)
-   Bước 4: Xây dựng mô hình hồi quy (Regression Model Building)

### Vì phạm vi của bài toán là thu thập và phân tích dữ liệu, nên chúng ta sẽ không đi sâu vào bước 4 (xây dựng mô hình hồi quy - Bài tập sau).


# Bước 1: Khai thác dữ liệu (Data Exploration)


### Import các thư viện cần thiết


In [ ]:
%pip install selenium

In [ ]:
%pip install webdriver_manager

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

## Đối với trang web cellphones.com.vn/mobile.html


### Đầu tiên ta sẽ set up, cào các Link cần thiết và kiểm thử từng bước trước khi tự động hóa toàn bộ quy trình


In [3]:
url = "https://cellphones.com.vn/mobile.html"
btn_show_more_cn = "button__show-more-product"

In [4]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(url)

Hiển thị toàn bộ sản phẩm trước khi lấy link


In [5]:
# Function to click all "Show More" buttons until none remain
def click_all_show_more(button_class, max_attempts=20):
    attempts = 0
    while attempts < max_attempts:
        try:
            # Find the button (adjust selector as needed)
            buttons = driver.find_elements(By.CLASS_NAME, button_class)
            if not buttons:
                print("No more 'Show More' buttons found")
                break
                
            # Scroll to button
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", buttons[0])
            time.sleep(1)
            
            # Click using JavaScript (bypasses visibility requirements)
            driver.execute_script("arguments[0].click();", buttons[0])
            
            print(f"Clicked 'Show More' button: {attempts+1} times")
            attempts += 1
            
            # Wait for new content to load
            time.sleep(3)
        except Exception as e:
            print(f"Error: {e}")
            break

In [6]:
# Wait until the button is clickable
wait = WebDriverWait(driver, 10)
button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, btn_show_more_cn)))
click_all_show_more(btn_show_more_cn, max_attempts=100)
product_card_cn = "div.product-info-container"

Clicked 'Show More' button: 1 times
Clicked 'Show More' button: 2 times
Clicked 'Show More' button: 3 times
Clicked 'Show More' button: 4 times
Clicked 'Show More' button: 5 times
Clicked 'Show More' button: 6 times
Clicked 'Show More' button: 7 times
Clicked 'Show More' button: 8 times
Clicked 'Show More' button: 9 times
Clicked 'Show More' button: 10 times
Clicked 'Show More' button: 11 times
Clicked 'Show More' button: 12 times
Clicked 'Show More' button: 13 times
Clicked 'Show More' button: 14 times
Clicked 'Show More' button: 15 times
Clicked 'Show More' button: 16 times
Clicked 'Show More' button: 17 times
Clicked 'Show More' button: 18 times
Clicked 'Show More' button: 19 times
Clicked 'Show More' button: 20 times
Clicked 'Show More' button: 21 times
Clicked 'Show More' button: 22 times
Clicked 'Show More' button: 23 times
Clicked 'Show More' button: 24 times
Clicked 'Show More' button: 25 times
Clicked 'Show More' button: 26 times
Clicked 'Show More' button: 27 times
Clicked 'S

KeyboardInterrupt: 

Xác định số lượng sản phẩm trên trang web


In [8]:
product_elements = driver.find_elements(By.CSS_SELECTOR, "div.product-info-container.product-item .product-info")
print(f"Total number of products found: {len(product_elements)}")

Total number of products found: 720


Xem thử thông tin của 1 sản phẩm


In [9]:
print(product_elements[0].text)

iPhone 16 Pro Max 256GB | Chính hãng VN/A
6.9 inches
256 GB
30.990.000đ
34.990.000đ
Giảm 11%
Smember giảm thêm đến 
307.000đ
Không phí chuyển đổi khi trả góp 0% qua thẻ tín dụng kỳ hạn 3-6 tháng


In [10]:
import concurrent.futures
from threading import Lock

# Thread-safe list and lock
product_links = []
link_lock = Lock()

# Function each thread will execute
def extract_product_link(product):
    try:
        # Check product price is not None, not empty, and not "Liên hệ" or "Giá liên hệ"
        price_element = product.find_element(By.CSS_SELECTOR, ".product__price--show").text.strip()
        if not price_element:
            print(f"Skipping product with empty price")
            return None
        if price_element in ["Liên hệ", "Giá liên hệ", ""]:
            print(f"Skipping product with price: {price_element}")
            return None
        
        # Find the link (adjust selector based on actual HTML structure)
        link_element = product.find_element(By.CSS_SELECTOR, "a.product__link")
        link = link_element.get_attribute("href")
        
        # Thread-safe append to the shared list
        with link_lock:
            product_links.append(link)
        
        return link
    except Exception as e:
        print(f"Error: {e} in product {product.text} - skipping")
        return None

# Number of threads to use
num_threads = 12  # Adjust based on your system's capabilities

# Use ThreadPoolExecutor to parallelize the work
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Submit all products to the thread pool
    futures = [executor.submit(extract_product_link, product) for product in product_elements]
    
    # Wait for all to complete
    concurrent.futures.wait(futures)

print(f"Found {len(product_links)} product links")
print(product_links[:5])

Found 720 product links
['https://cellphones.com.vn/iphone-16-pro-max.html', 'https://cellphones.com.vn/dien-thoai-samsung-galaxy-s25-ultra.html', 'https://cellphones.com.vn/xiaomi-14t-pro.html', 'https://cellphones.com.vn/samsung-galaxy-s24-ultra.html', 'https://cellphones.com.vn/dien-thoai-xiaomi-15.html']


### Sau khi lấy được các Link hợp lệ, ta tiến hành lưu vào file để đảm bảo không bị mất dữ liệu trong quá trình cào dữ liệu.


In [5]:
import csv

# Write the links to a CSV file
with open("product_links.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Product Link"])
    writer.writerows([[link] for link in product_links])

### Tiếp theo, ta tiến hành cào dữ liệu thô, ở đây do cấu trúc dữ liệu phức tạp nên sẽ lưu theo cấu trúc JSON


In [14]:
import json
import time
import traceback
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException, StaleElementReferenceException
from typing import Dict, Optional, Any, List

def execute_js(driver, js_script, *args):
    """
    Execute JavaScript with any number of arguments
    
    Parameters:
    - driver: Selenium WebDriver instance
    - js_script: JavaScript code to execute
    - *args: Any number of arguments to pass to the JavaScript
    
    Returns:
    - The result of the JavaScript execution
    """
    return driver.execute_script(js_script, *args)

def extract_product_common_info(driver: WebDriver, product_link: str) -> Dict[str, Any]:
    """
    Extract common product information from a product page with improved robustness.
    
    Parameters:
    - driver: Selenium WebDriver instance
    - product_link: URL of the product page
    
    Returns:
    - Dictionary containing product information
    """
    # Initialize result dictionary with default values
    product_data = {
        "url": product_link,
        "name": "",
        "price": "",
        "specifications": []
    }
    
    try:
        print(f"Extracting data from: {product_link}")
        
        # Loại bỏ chat model và các overlay nếu có
        execute_js(driver, """
            var elements = [
                document.getElementById('cs-live-chat'),
                document.querySelector('.csh-overlay'),
                document.querySelector('.chat-widget'),
                document.querySelector('.modal-overlay')
            ];
            elements.forEach(el => { if(el) el.remove(); });
        """)
        time.sleep(1)
        
        # CHIẾN LƯỢC 1: Lấy tên sản phẩm (thử nhiều selector)
        try:
            name_selectors = [
                ".box-product-name h1", 
                "a.item-linked.active strong",
            ]
            
            for selector in name_selectors:
                try:
                    name_element = driver.find_element(By.CSS_SELECTOR, selector)
                    name = name_element.text.strip()
                    if name:
                        product_data["name"] = name
                        print(f"Found product name: {name}")
                        if (name.strip() != ""):
                            break
                except (NoSuchElementException, StaleElementReferenceException):
                    continue
        except Exception as e:
            print(f"Error getting product name: {e}")
            
        # CHIẾN LƯỢC 2: Lấy giá sản phẩm (thử nhiều selector)
        try:
            price_selectors = [
                "a.item-linked.active span",
                ".product__price--show", 
                ".total-price",
            ]
            
            for selector in price_selectors:
                try:
                    price_element = driver.find_element(By.CSS_SELECTOR, selector)
                    price = price_element.text.strip()
                    if price:
                        product_data["price"] = price
                        print(f"Found product price: {price}")
                        if (price != "" and price != "0"):
                            break
                except (NoSuchElementException, StaleElementReferenceException):
                    continue
        except Exception as e:
            print(f"Error getting product price: {e}")
        
        # CHIẾN LƯỢC 3: Trích xuất thông số kỹ thuật qua nhiều cách
        specifications = []
        spec_extracted = False
        
        # Cách 1: Thử click vào button và lấy từ modal
        try:
            # Scroll xuống trang để button xuất hiện
            for scroll_position in [250, 500, 750, 1000, 1250]:
                try:
                    execute_js(driver, f"window.scrollTo(0, {scroll_position});")
                    time.sleep(0.5)
                    
                    # Thử tìm và click button bằng nhiều cách
                    button_clicked = False
                    
                    # Cách 1: Sử dụng selector trực tiếp
                    try:
                        button_selectors = [
                            ".button__show-modal-technical",
                            "button:contains('Xem cấu hình chi tiết')",
                        ]
                        
                        for selector in button_selectors:
                            try:
                                click_script = f"""
                                    var btn = document.querySelector('{selector}');
                                    if(btn) {{
                                        btn.scrollIntoView({{block: 'center'}});
                                        btn.click();
                                        return true;
                                    }}
                                    return false;
                                """
                                button_clicked = execute_js(driver, click_script)
                                if button_clicked:
                                    print(f"Clicked technical button with selector: {selector}")
                                    time.sleep(1.5)
                                    break
                            except:
                                continue
                    except Exception as e:
                        print(f"Error clicking button with selector: {e}")
                    
                    # Cách 2: Tìm bằng XPath với nội dung text
                    if not button_clicked:
                        try:
                            xpath_patterns = [
                                "//button[contains(text(), 'Xem cấu hình chi tiết')]",
                                "//a[contains(text(), 'Xem cấu hình chi tiết')]",
                                "//div[contains(text(), 'Xem cấu hình chi tiết')]",
                                "//span[contains(text(), 'Xem cấu hình chi tiết')]"
                            ]
                            
                            for xpath in xpath_patterns:
                                try:
                                    button = driver.find_element(By.XPATH, xpath)
                                    if button.is_displayed():
                                        execute_js(driver, "arguments[0].scrollIntoView({block: 'center'});", button)
                                        time.sleep(0.5)
                                        execute_js(driver, "arguments[0].click();", button)
                                        button_clicked = True
                                        print(f"Clicked technical button with xpath: {xpath}")
                                        time.sleep(1.5)
                                        break
                                except:
                                    continue
                        except Exception as e:
                            print(f"Error clicking button with xpath: {e}")
                    
                    if button_clicked:
                        # Đợi modal hiển thị
                        try:
                            modal_selectors = [
                                ".technical-content-modal",
                                ".modal-content",
                            ]
                            
                            modal_found = False
                            for selector in modal_selectors:
                                try:
                                    WebDriverWait(driver, 3).until(
                                        EC.visibility_of_element_located((By.CSS_SELECTOR, selector))
                                    )
                                    print(f"Modal found with selector: {selector}")
                                    modal_found = True
                                    
                                    # Trích xuất dữ liệu từ modal với JS
                                    js_script = """
                                    function extractSpecifications() {
                                        // Thử với cấu trúc modal cellphones.com.vn
                                        const ul = document.querySelector('ul.technical-content-modal');
                                        if (ul) {
                                            const items = Array.from(ul.querySelectorAll('li.technical-content-modal-item'));
                                            return items.map(item => {
                                                const titleElem = item.querySelector('p.title');
                                                const category = titleElem ? titleElem.innerText.trim() : "";
                                                const specDivs = item.querySelectorAll('div.modal-item-description > div');
                                                const details = {};
                                                
                                                specDivs.forEach(div => {
                                                    const keyElem = div.querySelector('p');
                                                    const valueElem = div.querySelector('div');
                                                    if(keyElem && valueElem) {
                                                        const key = keyElem.innerText.trim();
                                                        let value = valueElem.innerText.trim();
                                                        if(value.indexOf('\\n') !== -1) {
                                                            value = value.split('\\n').map(s => s.trim()).filter(s => s !== "");
                                                        }
                                                        details[key] = value;
                                                    }
                                                });
                                                return { category: category, details: details };
                                            });
                                        }
                                        
                                        // Thử với cấu trúc bảng thông thường
                                        const tables = document.querySelectorAll('table.specifications-table, table.tech-spec-table, .modal-content table');
                                        if (tables.length > 0) {
                                            const result = [];
                                            
                                            tables.forEach(table => {
                                                let currentCategory = "Thông số kỹ thuật";
                                                const categoryHeader = table.querySelector('th, caption, .spec-title');
                                                if (categoryHeader) {
                                                    currentCategory = categoryHeader.innerText.trim();
                                                }
                                                
                                                const details = {};
                                                const rows = table.querySelectorAll('tr');
                                                
                                                rows.forEach(row => {
                                                    const cells = row.querySelectorAll('td');
                                                    if (cells.length >= 2) {
                                                        const key = cells[0].innerText.trim();
                                                        let value = cells[1].innerText.trim();
                                                        
                                                        if(value.indexOf('\\n') !== -1) {
                                                            value = value.split('\\n').map(s => s.trim()).filter(s => s !== "");
                                                        }
                                                        
                                                        if (key && value) {
                                                            details[key] = value;
                                                        }
                                                    }
                                                });
                                                
                                                if (Object.keys(details).length > 0) {
                                                    result.push({ category: currentCategory, details: details });
                                                }
                                            });
                                            
                                            return result;
                                        }
                                        
                                        // Không tìm thấy dữ liệu nào
                                        return [];
                                    }
                                    return JSON.stringify(extractSpecifications());
                                    """
                                    
                                    json_data = execute_js(driver, js_script)
                                    extracted_specs = json.loads(json_data)
                                    
                                    if extracted_specs and len(extracted_specs) > 0:
                                        specifications = extracted_specs
                                        spec_extracted = True
                                        print(f"Extracted {len(extracted_specs)} specification categories from modal")
                                        break
                                except:
                                    continue
                            
                            if modal_found and spec_extracted:
                                break
                        except Exception as e:
                            print(f"Error extracting from modal: {e}")
                except Exception as e:
                    print(f"Error during modal interaction at scroll {scroll_position}: {e}")
                    
                if spec_extracted:
                    break
        except Exception as e:
            print(f"Error in modal extraction strategy: {e}")
        
        # Lưu thông số vào kết quả
        if specifications:
            product_data["specifications"] = specifications
            
    except Exception as e:
        print(f"Error during extraction: {traceback.format_exc()}")
    
    print(f"product name common data: ", product_data)
    return product_data

### Gọi hàm lấy dữ liệu và lưu dữ liệu vào file json


In [15]:
def getAndSaveProductInfo(driver: WebDriver, product_link: str) -> Dict[str, Any]:
    """
    Lấy thông tin sản phẩm từ trang product_link bằng cách cào thông tin common và variant, sau đó merge lại thành 1 dict.
    
    Returns:
    - Dictionary chứa thông tin sản phẩm, bao gồm:
         - url, specifications, variant_prices
    """
    driver.get(product_link)
    time.sleep(2)
    
    product_common_data = extract_product_common_info(driver, product_link)
    if product_common_data is None:
        product_common_data = {"url": product_link, "specifications": {}}
    
    file_name = product_link.split("/")[-1].replace(".html", "")
    print("product name common data: ", product_common_data)
    with open(f"json/{file_name}.json", "w", encoding="utf-8") as f:
        json.dump(product_common_data, f, indent=4, ensure_ascii=False)
        
    return product_common_data

Test hàm với 1 link sản phẩm ngẫu nhiên triong file product_links.csv


In [16]:
product_info = getAndSaveProductInfo(driver, "https://cellphones.com.vn/iphone-16-pro-max.html")

Extracting data from: https://cellphones.com.vn/iphone-16-pro-max.html
Found product name: iPhone 16 Pro Max 256GB | Chính hãng VN/A
Found product price: 30.990.000 đ
Clicked technical button with selector: .button__show-modal-technical
Modal found with selector: .technical-content-modal
Extracted 15 specification categories from modal
product name common data:  {'url': 'https://cellphones.com.vn/iphone-16-pro-max.html', 'name': 'iPhone 16 Pro Max 256GB | Chính hãng VN/A', 'price': '30.990.000 đ', 'specifications': [{'category': 'Màn hình', 'details': {'Kích thước màn hình': '6.9 inches', 'Độ phân giải màn hình': '2868 x 1320 pixels', 'Công nghệ màn hình': 'Super Retina XDR OLED', 'Tính năng màn hình': ['Dynamic Island', 'Màn hình Luôn Bật', 'Công nghệ ProMotion với tốc độ làm mới thích ứng lên đến 120Hz', 'Màn hình HDR', 'True Tone', 'Dải màu rộng (P3)', 'Haptic Touch', 'Tỷ lệ tương phản 2.000.000:1'], 'Tần số quét': '120Hz', 'Kiểu màn hình': 'Dynamic Island'}}, {'category': 'Cổng kết

In [17]:
import csv
product_links = []
with open("product_links.csv", "r") as f:
    reader = csv.reader(f)
    next(reader)  # Skip header
    for row in reader:
        product_links.append(row[0])
product_links[:5]

['https://cellphones.com.vn/iphone-16-pro-max.html',
 'https://cellphones.com.vn/dien-thoai-xiaomi-redmi-note-14.html',
 'https://cellphones.com.vn/dien-thoai-xiaomi-15.html',
 'https://cellphones.com.vn/dien-thoai-samsung-galaxy-s25.html',
 'https://cellphones.com.vn/oppo-reno10-pro-plus.html']

In [20]:
import os

# Lưu kết quả vào file JSON mà không ghi đè dữ liệu cũ
def save_results_append(results, filename="all_products.json"):
    try:
        all_data = []
        
        # Kiểm tra xem file đã tồn tại chưa
        if os.path.exists(filename):
            # Đọc dữ liệu từ file hiện có
            with open(filename, "r", encoding="utf-8") as f:
                try:
                    all_data = json.load(f)
                    print(f"Đọc được {len(all_data)} sản phẩm từ {filename}")
                except json.JSONDecodeError:
                    print(f"File {filename} không phải định dạng JSON hợp lệ. Sẽ tạo file mới.")
                    all_data = []
        
        # Kiểm tra xem all_data có phải là list không
        if not isinstance(all_data, list):
            print(f"Dữ liệu trong {filename} không phải là danh sách. Sẽ tạo danh sách mới.")
            all_data = []
        
        # Tìm các URL đã tồn tại để tránh trùng lặp
        existing_urls = {item.get("url", "") for item in all_data}
        new_items = 0
        
        # Thêm các mục mới chưa tồn tại
        for result in results:
            url = result.get("url", "")
            if url and url not in existing_urls:
                all_data.append(result)
                existing_urls.add(url)
                new_items += 1
        
        # Ghi toàn bộ dữ liệu vào file
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(all_data, f, indent=4, ensure_ascii=False)
        
        print(f"Đã thêm {new_items} sản phẩm mới vào {filename}")
        print(f"Tổng số sản phẩm hiện tại: {len(all_data)}")
        
    except Exception as e:
        print(f"Lỗi khi lưu {filename}: {str(e)}")

### Cuối cùng đây là hàm chính để chạy và cào dữ liệu, đảm bảo đã làm xong bước get Link và chạy các khai báo hàm rồi mới chạy được ở đây

-   Để giảm thời gian cào, ở đây nhóm sử dụng Đa luồng đa tuyến để cào dữ liệu. Lưu lại các Link lỗi để tiếp tục xử lí sau khi cào xong
-   Sử dụng 1 vài thư viện để log thông tin dễ theo dõi, debug trong quá trình cào dữ liệu


In [ ]:
import concurrent.futures
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from typing import Dict, List, Any
import time
import json
import os
import random
from tqdm import tqdm

def crawl_products_multithreaded(product_links: List[str], max_workers: int = 5, 
                                 batch_size: int = 10, delay_between_batches: int = 3) -> List[Dict[str, Any]]:
    """
    Cào dữ liệu sản phẩm đa luồng từ danh sách các liên kết sản phẩm.
    
    Nếu một liên kết gặp lỗi, sẽ được thêm vào cuối danh sách để xử lý lại tuần tự sau.
    
    Args:
        product_links: Danh sách các URL sản phẩm cần cào
        max_workers: Số lượng luồng tối đa chạy đồng thời
        batch_size: Kích thước mỗi batch để tránh quá tải
        delay_between_batches: Thời gian chờ giữa các batch (giây)
    
    Returns:
        Danh sách các dictionary chứa thông tin sản phẩm
    """
    # Đảm bảo thư mục json tồn tại
    os.makedirs("json", exist_ok=True)
    
    results = []
    retry_links = []  # Danh sách các liên kết cần thử lại
    
    # Chia danh sách sản phẩm thành các batch nhỏ hơn
    batches = [product_links[i:i+batch_size] for i in range(0, len(product_links), batch_size)]
    
    print(f"Chia thành {len(batches)} batch, mỗi batch {batch_size} sản phẩm")
    
    def worker(link):
        try:
            # Khởi tạo driver riêng cho mỗi thread
            options = Options()
            options.add_argument("--headless")  # Chạy Chrome ngầm không hiển thị giao diện (tiết kiệm tài nguyên)
            options.add_argument("--disable-gpu")  # Tắt GPU acceleration, giải quyết một số lỗi khi chạy headless
            options.add_argument("--no-sandbox")  # Tắt chế độ sandbox, cần thiết khi chạy trong môi trường Docker hoặc CI/CD
            options.add_argument("--disable-dev-shm-usage")  # Tránh lỗi tràn bộ nhớ shared memory trong Docker/container
            
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                                      options=options)
            
            try:
                # Random Sleep trước mỗi request để tránh bị chặn
                time.sleep(random.uniform(1, 3))
                # Gọi hàm getAndSaveProductInfo để cào dữ liệu
                result = getAndSaveProductInfo(driver, link)
                return result
            finally:
                # Đảm bảo đóng driver sau khi sử dụng
                driver.quit()
                
        except Exception as e:
            error_msg = f"Lỗi khi xử lý link {link}: {str(e)}"
            print(error_msg)
            return {"url": link, "error": str(e), "specifications": {}, "variant_prices": []}
    
    # Xử lý theo từng batch với đa luồng
    for batch_idx, batch in enumerate(batches):
        print(f"Đang xử lý batch {batch_idx+1}/{len(batches)}")
        batch_results = []
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(worker, link): link for link in batch}
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(batch), 
                              desc=f"Batch {batch_idx+1}"):
                link = futures[future]
                try:
                    result = future.result()
                    batch_results.append(result)
                except Exception as e:
                    # Nếu có lỗi ngoài dự tính, báo lỗi và tiếp tục
                    print(f"Lỗi khi lấy kết quả cho {link}: {str(e)}")
                    batch_results.append({"url": link, "error": str(e), "specifications": {}, "variant_prices": []})
        
        results.extend(batch_results)
        print(f"Đã cào {len(batch_results)}/{len(batch)} sản phẩm từ batch {batch_idx+1}")
        print(f"Tổng số sản phẩm đã cào: {len(results)}/{len(product_links)}")
        
        if batch_idx < len(batches) - 1:
            print(f"Chờ {delay_between_batches} giây trước khi xử lý batch tiếp theo...")
            time.sleep(delay_between_batches)
    
    # Xử lý các liên kết lỗi sau khi đa luồng bằng cách tuần tự
    # Nếu một liên kết lỗi, thêm nó vào cuối danh sách retry_links để thử lại sau
    retry_links = [result["url"] for result in results if result.get("error")]
    attempt = 1
    while retry_links:
        print(f"\nBắt đầu xử lý lại {len(retry_links)} liên kết lỗi (Lần thử: {attempt})")
        current_retry = retry_links.copy()
        retry_links = []  # reset danh sách lỗi
        
        for link in tqdm(current_retry, desc=f"Retry attempt {attempt}"):
            try:
                options = Options()
                options.add_argument("--headless")
                options.add_argument("--disable-gpu")
                options.add_argument("--no-sandbox")
                options.add_argument("--disable-dev-shm-usage")
                
                driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), 
                                          options=options)
                try:
                    # Cho phép load chậm hơn, không thêm độ trễ ngẫu nhiên
                    time.sleep(2)
                    result = getAndSaveProductInfo(driver, link)
                    # Cập nhật kết quả mới vào danh sách results:
                    # Tìm vị trí của kết quả cũ và thay thế
                    for idx, r in enumerate(results):
                        if r["url"] == link:
                            results[idx] = result
                            break
                finally:
                    driver.quit()
            except Exception as e:
                print(f"Lỗi khi xử lý lại link {link}: {str(e)}")
                # Nếu lỗi vẫn xảy ra, thêm link vào cuối danh sách để thử lại sau
                retry_links.append(link)
        if retry_links:
            print(f"Vẫn còn {len(retry_links)} liên kết lỗi, chờ 10 giây trước khi thử lại...")
            time.sleep(10)
        attempt += 1

    # Lưu tất cả kết quả vào một file JSON
    try:
        with open("all_products.json", "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4, ensure_ascii=False)
        print(f"Đã lưu tất cả {len(results)} kết quả vào all_products.json")
    except Exception as e:
        print(f"Lỗi khi lưu all_products.json: {str(e)}")
    
    return results

# Ví dụ sử dụng:
if __name__ == "__main__":
    # product_links cần được định nghĩa trước đó (ví dụ: danh sách các URL sản phẩm)
    # Ví dụ: product_links = ["http://example.com/product1", "http://example.com/product2", ...]
    results = crawl_products_multithreaded(
        product_links=product_links[:500],
        max_workers=5,      # Giảm xuống 2 luồng
        batch_size=5,       # Giảm xuống 2 sản phẩm/batch
        delay_between_batches=10  # Tăng đợi giữa các batch
    )
    
    print(f"Hoàn thành cào dữ liệu cho {len(results)} sản phẩm")


Chia thành 100 batch, mỗi batch 5 sản phẩm
Đang xử lý batch 1/100


Batch 1:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data from: https://cellphones.com.vn/dien-thoai-xiaomi-redmi-note-14.html
Extracting data from: https://cellphones.com.vn/dien-thoai-samsung-galaxy-s25.html
Extracting data from: https://cellphones.com.vn/dien-thoai-xiaomi-15.html
Extracting data from: https://cellphones.com.vn/iphone-16-pro-max.html
Extracting data from: https://cellphones.com.vn/oppo-reno10-pro-plus.html
Found product name: Xiaomi Redmi Note 14 6GB 128GB
Found product price: 4.590.000 đ
Found product name: Samsung Galaxy S25 256GB
Found product price: 19.590.000 đ
Found product name: Xiaomi 15 5G 12GB 256GB
Found product price: 21.990.000 đ
Found product name: iPhone 16 Pro Max 256GB | Chính hãng VN/A
Found product price: 30.990.000 đ
Found product name: OPPO Reno10 Pro+ 5G 12GB 256GB
Found product price: 10.990.000 đ
Clicked technical button with selector: .button__show-modal-technical
Clicked technical button with selector: .button__show-modal-technical
Clicked technical button with selector: .button__sh

Batch 1: 100%|██████████| 5/5 [00:22<00:00,  4.48s/it]


Đã cào 5/5 sản phẩm từ batch 1
Tổng số sản phẩm đã cào: 5/500
Chờ 10 giây trước khi xử lý batch tiếp theo...
Đang xử lý batch 2/100


...

Batch 76:  60%|██████    | 3/5 [00:19<00:09,  4.60s/it]

Modal found with selector: .technical-content-modal
Extracted 7 specification categories from modal
product name common data:  {'url': 'https://cellphones.com.vn/oppo-find-x7-pro.html', 'name': 'OPPO Find X7 Pro', 'price': 'Giá Liên Hệ', 'specifications': [{'category': 'Màn hình', 'details': {'Kích thước màn hình': '6.7 inches', 'Công nghệ màn hình': 'LTPO OLED', 'Độ phân giải màn hình': '1440 x 3216 pixels (QHD+)', 'Tính năng màn hình': 'Tần số quét 120 Hz'}}, {'category': 'Camera sau', 'details': {'Camera sau': '50 MP + 50 MP + 13 MP'}}, {'category': 'Camera trước', 'details': {'Camera trước': '32 MP'}}, {'category': 'Vi xử lý & đồ họa', 'details': {'Chipset': 'Snapdragon 8+ Gen 3'}}, {'category': 'Giao tiếp & kết nối', 'details': {'Công nghệ NFC': 'Có'}}, {'category': 'RAM & lưu trữ', 'details': {'Dung lượng RAM': '12 GB', 'Bộ nhớ trong': '256 GB'}}, {'category': 'Pin & công nghệ sạc', 'details': {'Pin': '5.200 mAh', 'Công nghệ sạc': 'Sạc nhanh'}}]}
product name common data:  {'url'

Batch 76:  80%|████████  | 4/5 [00:21<00:03,  3.69s/it]

Extracting data from: https://cellphones.com.vn/vivo-iqoo-10-pro.html
Found product name: ViVo iQOO 10 Pro
Found product price: Giá Liên Hệ
Clicked technical button with selector: .button__show-modal-technical
Modal found with selector: .technical-content-modal
Extracted 12 specification categories from modal
product name common data:  {'url': 'https://cellphones.com.vn/vivo-iqoo-10-pro.html', 'name': 'ViVo iQOO 10 Pro', 'price': 'Giá Liên Hệ', 'specifications': [{'category': 'Màn hình', 'details': {'Kích thước màn hình': '6.78 inches', 'Công nghệ màn hình': 'AMOLED', 'Độ phân giải màn hình': '1440 x 3200 pixels (QHD+)', 'Tính năng màn hình': '1 tỉ màu, 120Hz, HDR10+, 500 nits (typ)'}}, {'category': 'Camera sau', 'details': {'Camera sau': ['Camera góc rộng: 50 MP, f/1.9, PDAF', 'Camera tele: 14.6 MP, f/2.2, zoom quang 3x', 'Camera góc siêu rộng: 50 MP, f / 2.3, 150˚'], 'Quay video': ['8K@30fps, 4K@24/30/60fps, 1080p@ 30/60fps', 'Con quay hồi chuyển EIS'], 'Tính năng camera': 'Đèn LED k

Batch 76:  80%|████████  | 4/5 [00:35<00:08,  8.87s/it]


### Đã hoàn thành bước cào dữ liệu, đối với mobilecity, workflow cào dữ liệu sẽ tương tự. Nhưng vì cấu trúc HTML và một vài quy ước ở 2 trang khác nhau nên sẽ tách file riêng "craw_data_2.ipynb"
